In [1]:
import pyvista as pv
# import topogenesis  as tp
import numpy as np
from honeybee_plus.hbsurface import HBSurface
from honeybee_plus.radiance.recipe.solaraccess.gridbased import SolarAccessGridBased
from honeybee_plus.radiance.sky.certainIlluminance import CertainIlluminanceLevel
from honeybee_plus.radiance.recipe.pointintime.gridbased import GridBased
from honeybee_plus.radiance.analysisgrid import AnalysisGrid
from honeybee_plus.radiance.parameters.rtrace import LowQuality, RtraceParameters
import os
import ladybug as lb
import pandas as pd

cwd = os.getcwd()
src = os.path.dirname(cwd)

Path to radiance is set to: /usr/local/radiance
Path to perl is set to: /usr/bin


In [2]:
material_glass = {'modifier': 'void', 
                 'type': 'glass', 
                 'name': 'generic_glass', 
                 'r_transmittance': 0.6, 
                 'g_transmittance': 0.6, 
                 'b_transmittance': 0.6, 
                 'refraction_index': 1.52
                }
material_plastic = {'modifier': 'void', 
                   'type': 'plastic', 
                   'name': 'generic_plastic', 
                   'r_reflectance': 0.2, 
                   'g_reflectance': 0.2, 
                   'b_reflectance': 0.2, 
                   'specularity': 0.0, 
                   'roughness': 0.0}
def mesh_to_hbsurface(mesh, s_type, s_name, mat):
    hb_surfaces = []
    face_list = list(mesh.faces)
    e = 0
    while e < len(face_list):
        v_count = face_list[e]
        vertices = []
        for v in range(v_count):
            e += 1
            vertices.append(list(mesh.points[face_list[e]]))
        srf_dict = {
            "name": s_name,
            "vertices": vertices, # [[(x, y, z), (x1, y1, z1), (x2, y2, z2)]],
            "surface_material": mat,
            "surface_type": s_type  # 0: wall, 5: window 
            # TODO: look for the lables of ceiling and floor
            # TODO: check if radiance need unique names for surfaces!
            }
        srf_dict["surface_material"]["name"] = s_name
        hbsrf = HBSurface.from_json(srf_dict)
        hb_surfaces.append(hbsrf)
        e += 1
    return hb_surfaces

In [3]:
# import all external geometry with pyvista
wal_filepath = os.path.join(src, "data", "test_room_walls_02.obj")
wal_mesh = pv.read(wal_filepath)
fen_filepath = os.path.join(src, "data", "test_room_fens_02.obj")
fen_mesh = pv.read(fen_filepath)

con_buildings_filepath = os.path.join(src, "data", "test_context_01_buildings.obj")
con_buildings_mesh = pv.read(con_buildings_filepath)
con_ground_filepath = os.path.join(src, "data", "test_context_01_ground.obj")
con_ground_mesh = pv.read(con_ground_filepath)
con_sky_filepath = os.path.join(src, "data", "test_context_01_sky.obj")
con_sky_mesh = pv.read(con_sky_filepath)
con_vegetation_filepath = os.path.join(src, "data", "test_context_01_vegetation.obj")
con_vegetation_mesh = pv.read(con_vegetation_filepath)

# construct honeybee surfaces from the mesh
wal_hb_surf = mesh_to_hbsurface(wal_mesh, 0, "wall", material_plastic)
fen_hb_surf = mesh_to_hbsurface(fen_mesh, 5, "window", material_glass)
con_buildings_hb_surf = mesh_to_hbsurface(con_buildings_mesh, 0, "context_buildings", material_plastic)
con_ground_hb_surf = mesh_to_hbsurface(con_ground_mesh, 0, "context_ground", material_plastic)
con_sky_hb_surf = mesh_to_hbsurface(con_sky_mesh, 0, "context_sky", material_plastic)
con_vegetation_hb_surf = mesh_to_hbsurface(con_vegetation_mesh, 0, "context_vegetation", material_plastic)

hb_surfaces = wal_hb_surf + fen_hb_surf + con_buildings_hb_surf + con_ground_hb_surf + con_sky_hb_surf + con_vegetation_hb_surf
# TODO: look into tagging system for out_view to specify the "external environment"

# print(hb_surfaces)
# print(type(mesh))
# print(mesh.points)
# print(mesh.faces)

In [4]:
# generate grid of test points
test_pts = [(1, 1, 1),(6, 1, 1),(1, 6, 1),(1, 1, 6)]
test_vec = [(0, 0, 1),(0, 0, 1),(0, 0, 1),(0, 0, 1)]
analysis_grid = AnalysisGrid.from_points_and_vectors(test_pts, test_vec, 'test_grid')

# generate the sky
sky = CertainIlluminanceLevel(illuminance_value=2000,sky_type=1)
print(CertainIlluminanceLevel.__doc__)

# put the recipe together # note simulation type is changed
rp = GridBased(sky=sky, analysis_grids=(analysis_grid,), simulation_type=2,
               hb_objects=hb_surfaces)

Uniform CIE sky based on illuminance value.

    Attributes:
        illuminance_value: Desired illuminance value in lux
        sky_type: An integer between 0..1 to indicate CIE Sky Type.
            [0] cloudy sky, [1] uniform sky (default: 0)
        suffix: An optional suffix for sky name. The suffix will be added at the
            end of the standard name. Use this input to customize the new and
            avoid sky being overwritten by other skymatrix components.
    Usage:

        sky = CertainIlluminanceLevel(1000)
        sky.execute("c:/ladybug/1000luxsky.sky")
    
Found 6386 opaque surfaces.
Found 1 fenestration surfaces.
Found 0 window-groups.


In [5]:
# rp.simulation_type = 1
print(rp.simulation_type)
# check the -n flag for the number of parallel proceses, find the number of cpu cores here: https://askubuntu.com/questions/724228/how-to-find-the-number-of-cpu-cores-including-virtual   https://superuser.com/questions/1101311/how-many-cores-does-my-mac-have
rad_param = RtraceParameters(0)
print(rad_param)
rad_param.ambient_bounces = 0
print(rad_param)
rad_param.output_spec = "s"
print(rad_param)
# update the radiance parameters
print(rp.radiance_parameters)
rp.radiance_parameters = rad_param

# write simulation to folder
batch_file = rp.write(
    target_folder='.', project_name='external_context_view')

# print the radiance commands
for i,com in enumerate(rp._commands):
    print(i, com)

# change the commands
#rp._commands[3] = "/usr/local/radiance/bin/rtrace -os -aa 0.25 -ab 0 -ad 512 -ar 16 -as 128 -dc 0.25 -dj 0.0 -dp 64 -ds 0.5 -dr 0 -dt 0.5 -e error.txt -h -I -lr 4 -lw 0.05 -ss 0.0 -st 0.85 external_context_view.oct < external_context_view.pts > result/external_context_view.res"

2
-aa 0.25 -ab 2 -ad 512 -ar 16 -as 128 -dc 0.25 -dj 0.0 -dp 64 -ds 0.5 -dr 0 -dt 0.5 -lr 4 -lw 0.05 -ss 0.0 -st 0.85
-aa 0.25 -ab 0 -ad 512 -ar 16 -as 128 -dc 0.25 -dj 0.0 -dp 64 -ds 0.5 -dr 0 -dt 0.5 -lr 4 -lw 0.05 -ss 0.0 -st 0.85
-aa 0.25 -ab 0 -ad 512 -ar 16 -as 128 -dc 0.25 -dj 0.0 -dp 64 -ds 0.5 -dr 0 -dt 0.5 -lr 4 -lw 0.05 -os -ss 0.0 -st 0.85
-aa 0.25 -ab 2 -ad 512 -ar 16 -as 128 -dc 0.25 -dj 0.0 -dp 64 -ds 0.5 -dr 0 -dt 0.5 -lr 4 -lw 0.05 -ss 0.0 -st 0.85
Writing recipe contents to: ./external_context_view/gridbased
0 #!/usr/bin/env bash



cd ./external_context_view/gridbased

1 /usr/local/radiance/bin/gensky 9 21 12 -u -B 11.1731843575419 > sky/CertainIlluminanceLevel_2000.sky
2 /usr/local/radiance/bin/oconv -f sky/CertainIlluminanceLevel_2000.sky sky/groundSky.rad scene/opaque/external_context_view..opq.mat scene/opaque/external_context_view..opq.rad scene/glazing/external_context_view..glz.mat scene/glazing/external_context_view..glz.rad > external_context_view.oct
3 /usr

In [ ]:
# run the simulation
rp.run(batch_file, debug=False)

# https://www.radiance-online.org/learning/documentation/manual-pages/pdfs/rmtxop.pdf
# Eleonora: rmtxop multiplies the daylight coefficient matrices with the sky vectors. This is necessary for getting illuminance from daylight coefficients computed w.r.t. rgb components. 
# Sun Access of points, and how to interpret them https://floyd.lbl.gov/radiance/refer/usman2.pdf

In [ ]:
# results - in this case it will be an analysis grid
result = rp.results()[0]
dfs = []

# print(result.result_files)
for i, ap in enumerate(result._analysis_points):
    data_dict = ap._values[0][0]
    print(data_dict)
    # df = pd.DataFrame(data_dict)
    # dft = df.T
    # dft.columns = [i, i+1]
    # dfs.append(dft[i])

# df = pd.concat(dfs, axis=1)
# print(df)
# print(sum(df[0])/4460.0)
# print(sum(df[1])/4460.0)
# print(sum(df[2])/4460.0)
# print(sum(df[3])/4460.0)
# df = pd.DataFrame({
#   'IX': vol_3d_ind_flat[:, 0],
#   'IY': vol_3d_ind_flat[:, 1],
#   'IZ': vol_3d_ind_flat[:, 2],
#   'value': vol_flat,
#})
# print result with legend
# leg = ["Daylight autonomy", "Continious daylight autonomy", "Useful daylight illuminance", "Less than UDI", "More than UDI"]
# for l, d in zip(leg, result.annual_metrics(da_threshhold=1)):
#    print(l, d)

# extract the points of interest from the recipe
result_points = [[p[0], p[1], p[2]] for p in result.points]

In [ ]:
# Plot using the ITKplotter
pl = pv.PlotterITK()
# add the external wall mesh
pl.add_mesh(wal_mesh, smooth_shading=True)
# add the external fens mesh, transparrent blue
pl.add_mesh(fen_mesh, smooth_shading=True, opacity=0.3, color='#c1e2ff')
# add the context mesh, transparrent grey
pl.add_mesh(con_buildings_mesh, smooth_shading=True, opacity=0.3)
pl.add_mesh(con_ground_mesh, smooth_shading=True, opacity=0.3)
pl.add_mesh(con_sky_mesh, smooth_shading=True, opacity=0.3)
pl.add_mesh(con_vegetation_mesh, smooth_shading=True, opacity=0.3)
# add points of interest : blue
pl.add_points(pv.PolyData(result_points), color='#2499ff')
# add sun points : orange
# pl.add_points(pv.PolyData(np.array(sun_vectors) * -120), color='#ffa500')
# show the plot
pl.show(True)